In [20]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk

In [2]:
news = pd.read_csv('newzy.csv' , sep = '|')

In [3]:
news.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
0,39,Google News,"50 Years on, Finding Profit in 'Truth' on JFK ...","Washington Post50 Years on, Finding Profit in ...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
1,38,Google News,"Nearly 10 years later, families and officials ...","Washington PostNearly 10 years later, families...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
2,37,Google News,"Cuccinelli, running hard, has been hobbled in ...","Washington PostCuccinelli, running hard, has b...",11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
3,36,Google News,Chuck Schumer endorses Hillary Clinton for Pre...,New York Daily NewsChuck Schumer endorses Hill...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...
4,35,Google News,Sec. Kerry issues apparent warning to Iran - C...,Ahram OnlineSec. Kerry issues apparent warning...,11/03/2013,http://news.google.com/news/url?sa=t&fd=R&usg=...


In [10]:
news.doc_source.unique()

array(['Google News', 'PowerLine', 'Reuters', 'Guardian', 'Daily Kos',
       'Drudge Report', 'Real Clear Politics', 'NPR', 'Fox',
       'New York Times', 'Breitbart', 'CNN', 'US News', 'UPI Latest',
       'Politico Magazine', nan], dtype=object)

In [12]:
df = news[news.doc_source.isin(['PowerLine', 'Politico Magazine'])]

In [18]:
df.to_csv('df.csv')

In [19]:
df.head()

,doc_id,doc_source,doc_title,doc_content,doc_date,doc_url
25,48,PowerLine,These Zins Are a Dusi,(Steven Hayward) I’ve been falling behind on m...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
26,49,PowerLine,NY Times Editorial Board Hacked by The Onion,(John Hinderaker) At least I think it was the ...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
27,50,PowerLine,The line Obama crossed,(Scott Johnson) Obamacare was enacted on a fou...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
28,51,PowerLine,Sham/Wow,(Scott Johnson) It’s unfair to compare and con...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...
29,52,PowerLine,A Quarterback Controversy With a Political Twist?,(John Hinderaker) Public Policy Polling ran a ...,11/03/2013,http://feeds.powerlineblog.com/~r/powerlineblo...


In [33]:
OHCO = ['doc_source', 'doc_id', 'sent_num', 'token_num']
#date - y m w, weekly cycles
#parse authors from content


In [40]:
LIBRARY = df[['doc_source', 'doc_id', 'doc_content', 'doc_date']]
LIBRARY = LIBRARY.reset_index().set_index(OHCO[:2]).drop(columns = 'index')

LIBRARY.head() #doc table??

doc_content  \
doc_source doc_id                                                      
PowerLine  48      (Steven Hayward) I’ve been falling behind on m...   
           49      (John Hinderaker) At least I think it was the ...   
           50      (Scott Johnson) Obamacare was enacted on a fou...   
           51      (Scott Johnson) It’s unfair to compare and con...   
           52      (John Hinderaker) Public Policy Polling ran a ...   

                     doc_date  
doc_source doc_id              
PowerLine  48      11/03/2013  
           49      11/03/2013  
           50      11/03/2013  
           51      11/03/2013  
           52      11/03/2013

In [66]:
LIBRARY.doc_content.iloc[0]

'(Steven Hayward) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.\xa0 So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.\xa0 The Dusi family’s Zinfandel vineyards go back nearly a century. \xa0For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.\xa0 The Dusi family seldom made their own wine—until now.\xa0 And here’s the four-minute storyboard for it.\xa0 (Footnote: I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”\xa0 The time has come.) if(typeof(jQuery)=="function"){(function($){$.fn.fitVids=function(){}})(jQuery)};jwplayer(\'jwplayer-2\').setup({"aspectratio":null,"width":580,"height":326,"primary":"html5","logo":{"file":"http://www.powerlineblog.com/pl_vid_bug_BW.png","hide":false,"link":"http://www.pow

In [60]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.doc_content\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(str(x))))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x))) # Discards stuff in between
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [56]:
LIBRARY.doc_content

doc_source  doc_id 
PowerLine   48         (Steven Hayward) I’ve been falling behind on m...
            49         (John Hinderaker) At least I think it was the ...
            50         (Scott Johnson) Obamacare was enacted on a fou...
            51         (Scott Johnson) It’s unfair to compare and con...
            52         (John Hinderaker) Public Policy Polling ran a ...
                                             ...                        
            1026329    (Steven Hayward) I think it was Tom Bethell, b...
            1026330    (Paul Mirengoff) The Trump Department of Justi...
            1026331    (John Hinderaker) No one in public life has fo...
            1026332    (John Hinderaker) Today the 2nd Circuit Court ...
            1026333    (John Hinderaker) Has anyone in American histo...
Name: doc_content, Length: 13400, dtype: object

In [61]:
LIBRARY.doc_content.apply(lambda x: pd.Series(nltk.sent_tokenize(str(x))))

0    \
doc_source doc_id                                                       
PowerLine  48       (Steven Hayward) I’ve been falling behind on m...   
           49       (John Hinderaker) At least I think it was the ...   
           50       (Scott Johnson) Obamacare was enacted on a fou...   
           51       (Scott Johnson) It’s unfair to compare and con...   
           52       (John Hinderaker) Public Policy Polling ran a ...   
...                                                               ...   
           1026329  (Steven Hayward) I think it was Tom Bethell, b...   
           1026330  (Paul Mirengoff) The Trump Department of Justi...   
           1026331  (John Hinderaker) No one in public life has fo...   
           1026332  (John Hinderaker) Today the 2nd Circuit Court ...   
           1026333  (John Hinderaker) Has anyone in American histo...   

                                                                  1    \
doc_source doc_id                                                       
PowerLine  48                       So here’s a quick catch up guide.   
           49       Someone, anyway, got access to the Times Edito...   
           50       They are the centerpiece of our Lies of Obamac...   
           51                             James Taranto explains why.   
           52       The political results were predictably dreary;...   
...                                                               ...   
           1026329  That illuminating question has become suddenly...   
           1026330  Yesterday, pursuant to this policy, the DOJ fi...   
           1026331  It later came out that the government’s respon...   
           1026332  v. U.S. Department of Justice, upholding the T...   
           1026333                                 Lincoln, possibly.   

                                                                  2    \
doc_source doc_id                                                       
PowerLine  48       Old timers of Paso Robles wines all know the D...   
           49       That, they say, violates President Obama’s ple...   
           50       Today Kyle Smith devotes a New York Post colum...   
           51       Whereas a scammer selling shoddy goods is brea...   
           52       But for some reason, PPP also asked some sport...   
...                                                               ...   
           1026329  Bethell’s astute point, 30 years ago, is being...   
           1026330  The case is on appeal from a district court ru...   
           1026331  President Trump understands, no doubt, that th...   
           1026332                       The court’s opinion is here.   
           1026333                It is hard to think of anyone else.   

                                                                  3    \
doc_source doc_id                                                       
PowerLine  48       The Dusi family’s Zinfandel vineyards go back ...   
           49           Mr. Obama clearly misspoke when he said that.   
           50       Smith’s column puts an exclamation point on th...   
           51       “Thus there is no evident legal recourse for t...   
           52                                          Let’s back up.   
...                                                               ...   
           1026329  This can be proven with Euclidian precision wi...   
           1026330         We discussed the decision  here and  here.   
           1026331  The Trump administration took prompt steps to ...   
           1026332  The back story includes three prior Court of A...   
           1026333  Among other things, it is an article of faith ...   

                                                                  4    \
doc_source doc_id                                                       
PowerLine  48       For decades, if you wanted to find the best Zi...   
           49                                  

In [48]:
type(LIBRARY.doc_content)

pandas.core.series.Series

In [44]:
TOKEN = tokenize(LIBRARY, ws=False)

TypeError: expected string or bytes-like object

In [ ]:
TOKEN

In [ ]:
VOCAB

In [15]:
df.doc_content.iloc[1]

'(John Hinderaker) At least I think it was the Onion. Someone, anyway, got access to the Times Editorial Board computers and turned out this parody editorial on Obamacare: Congressional Republicans have stoked consumer fears and confusion with charges that the health care reform law is causing insurers to cancel existing policies and will force many people to pay substantially higher premiums next year for coverage they don’t want. That, they say, violates President Obama’s pledge that if you like the insurance you have, you can keep it. Mr. Obama clearly misspoke when he said that. HaHaHaHaHa! President Obama misspoke 27 times! And curiously enough, when he misspoke it was the centerpiece of his argument for his supposedly signature legislative achievement! Even the Times editorial board wouldn’t try to sell that one; it’s got to be the Onion. By law, insurers cannot continue to sell policies that don’t provide the minimum benefits and consumer protections required as of next year. So

In [12]:
news.doc_content[1]

"Washington PostNearly 10 years later, families and officials still search for answers in SC bike ...Fox NewsSPARTANBURG, S.C. – One of South Carolina's most heinous killings remain unsolved after almost 10 years. Four people were killed inside a motorcycle shop in Spartanburg County on Nov. 6, 2003. Investigators have followed more than 750 leads but\xa0...10 years on, SC bike shop killings still unsolvedHouston Chronicleall 9 news articles\xa0»"

In [13]:
news.doc_date.unique()

array(['11/03/2013', '11/04/2013', '11/05/2013', ..., '02/25/2020',
       '02/26/2020', '02/27/2020'], dtype=object)